In [ ]:
using Pkg
pkg"activate ."
#=
pkg"add NLPModels"
pkg"add SolverTools"
pkg"add CUTEst"
pkg"add SolverBenchmark"
pkg"add Plots"
pkg"add PyPlot"
=#

In [ ]:
pkg"status"

# Introdução à Métodos Computacionais de Otimização

**Def. (Algoritmo):** Um algoritmo é uma coleção de instruções para realizar alguma tarefa específica. Segundo Donald Knuth (The Art of Computer Programming, v.1), ele deve satisfazer as seguintes condições, parafraseadas aqui:
- **Finitude:** O algoritmo deve acabar em tempo finito;
- **Bem definido:** As intruções devem ser claras e sem ambiguidade;
- **Entrada:** O algoritmo tem zero ou mais entradas, que são valores determinados antes do algoritmo começar. Essas entradas são especificados a partir de conjuntos de objetos;
- **Saída:** O algoritmo tem uma ou mais saídas, que são quantidades relacionadas com as entradas;
- **Eficácia/Computabilidade:** As operações feitas no algoritmo devem ser suficientemente básicas para que a princípio possam ser executadas por uma pessoa num espaço finito e tempo finito com papel e caneta.

Um **método** é menos restrito, e uma **implementação** é a realização computacional de um algoritmo.

Pode-se dizer que, dado um problema, existem vários métodos para resolvê-lo, para cada método, existem vários algoritmos que o usam, para cada algoritmo, várias implementações.

# Condições de paradas

Um algoritmo de otimização será um algoritmo com o objetivo de encontrar pontos críticos de um problema informado pelo usuário. Alguns algoritmos podem verificar condições de segunda ordem, mas em geral estaremos olhando apenas para as condições de primeira ordem. No caso irrestrito continuamente diferenciável, isso quer dizer $\nabla f(\overline{x}) = 0$. Essa condição precisa ser relaxada para obtermos uma condição razoável de parada de sucesso, isto é, uma condição que diz que o ponto encontrado está suficientemente próximo de ser um ponto crítico do problema. Para tanto, utilizaremos uma *tolerância* para a condição de primeira ordem: Buscaremos um ponto onde o gradiente está suficientemente próximo de zero, por exemplo verificando
$\Vert \nabla f(x_k)\Vert < \varepsilon$ ou
$\Vert \nabla f(x_k)\Vert < \varepsilon \Vert \nabla f(x_0)\Vert$, ou ainda, uma combinação das duas:
$$\Vert \nabla f(x_k)\Vert < \varepsilon_a + \varepsilon_r \Vert \nabla f(x_0)\Vert.$$

Os métodos de otimização costumam ter alguma garantia de convergência perto da solução, ou em alguns casos, de gerar pontos de acumulação críticos. No entanto, pode acontecer de o método demorar demais na busca de uma solução, ou de encontrar um obstáculo que não pode ultrapassar. Para evitar que o seu programa tenha um *bug*, é preciso uma combinação de entendimento computacional e teórico do algoritmo para criar condições de parada adicionais. As mais comuns estão a seguir:

- Máximo de iterações, de avaliação de funções, de tempo decorrido, etc.
- Insatisfação de alguma condição teórica para o funcionamento do método, e.g., positividade da Hessiana em Newton, geração de uma direção que não seja de descida;
- Acontecimento computacional que não deveria acontecer na matemática exata, mas que ocorre devido ao uso de ponto flutuante, .e.g, má condicionamento da Hessiana, o passo de Armijo vira zero, divisão por algum número muito próximo de zero, a direção fica muito perto de ser ortogonal;
- Acontecimentos puramente computacionais, e.g., falta de memória;

Muitos problemas só são percebidos após a implementação é iniciada, por isso é preciso ficar atento e implementar testes para o seu código.

## Indicador de Saída - Status

O comum nessas situações de parada é indicar com alguma variável o que aconteceu. No passado, o costume era retornar $0$ se tudo correu bem, $>0$ para paradas previstas, e $<0$ para exceções.
Hoje em dia podemos retornar texto ou símobolos descrevendo a saída, por exemplo, `"sucesso"`, `"máximo de iterações"`, `"máximo de tempo"`, etc.

# JuliaSmoothOptimizers

![JuliaSmoothOptimizers](jso.png)

O [JuliaSmoothOptimizers](https://github.com/JuliaSmoothOptimizers) é um framework de desenvolvimento de solvers em Julia. Ele ajuda a definir um modelo, criar métodos de otimização, compará-los, e gerar gráficos e tabelas. Vamos apresentar as partes mais básicas do framework para não sobrecarregar.

## NLPModels.jl

In [ ]:
using NLPModels

f(x) = (x[1] - 1)^2 + 4 * (x[2] - x[1]^2)^2
x₀ = [-1.2; 1.0]
nlp = ADNLPModel(f, x₀)

In [ ]:
obj(nlp, x₀)

In [ ]:
obj(nlp, [0.0; 2.0])

In [ ]:
grad(nlp, x₀)

In [ ]:
grad(nlp, [1.0; 1.0])

In [ ]:
hess(nlp, x₀)

In [ ]:
hess(nlp, [1.0; 1.0])

In [ ]:
using LinearAlgebra

Symmetric(hess(nlp, x₀), :L)

In [ ]:
Hx = Symmetric(hess(nlp, [1.0; 1.0]), :L)

In [ ]:
eigen(Hx).values

## Método de Newton Puro

1. Dados $x_0$, $\varepsilon > 0$, $k = 0$, $k_\max$.
2. Enquanto $\Vert\nabla f(x_k)\Vert > \varepsilon$
    1. Calcule $d_k$ resolvendo o sistema $\nabla^2 f(x_k) d = -\nabla f(x_k)$
    2. Calcule $x_{k+1} = x_k + d_k$
    3. $k = k + 1$
    4. Teste outras condições de parada e vá à 4 se alguma for satisfeita
3. Fim do Enquanto
4. Saída: $x_k$, $f(x_k)$, $\Vert\nabla f(x_k)\Vert$, Tempo, Iterações, status

In [ ]:
using Plots
pyplot(size=(400,400))

In [ ]:
xg = range(-2.0, 2.0, length=100)
yg = range(-1.0, 3.0, length=100)
contour(xg, yg, (x,y) -> obj(nlp, [x;y]), levels=100, leg=false)

In [ ]:
g(x) = grad(nlp, x)
H(x) = Symmetric(hess(nlp, x), :L)

In [ ]:
g(ones(2))

In [ ]:
x = nlp.meta.x0

In [ ]:
d = -H(x) \ g(x)

In [ ]:
contour(xg, yg, (x,y) -> obj(nlp, [x;y]), levels=50, leg=false)
scatter!([x[1]], [x[2]], c=:red, ms=3, leg=false)
plot!([x[1], x[1] + d[1]], [x[2], x[2] + d[2]], c=:blue, l=:arrow)
scatter!([x[1] + d[1]], [x[2] + d[2]], c=:blue, ms=3)

In [ ]:
p = contour(xg, yg, (x,y) -> obj(nlp, [x;y]), levels=50, leg=false)
x = copy(nlp.meta.x0)
scatter!([x[1]], [x[2]], c=:red, ms=3, leg=false)
d = -H(x) \ g(x)
plot!([x[1], x[1] + d[1]], [x[2], x[2] + d[2]], c=:blue, l=:arrow)
scatter!([x[1] + d[1]], [x[2] + d[2]], c=:blue, ms=3)
while norm(d) > 1e-4
    x += d
    d = -H(x) \ g(x)
    plot!([x[1], x[1] + d[1]], [x[2], x[2] + d[2]], c=:blue, l=:arrow)
    scatter!([x[1] + d[1]], [x[2] + d[2]], c=:blue, ms=3)
end
p

In [ ]:
using SolverTools

function newton_puro(nlp :: AbstractNLPModel;
                     tol = 1e-6, kmax = 10000, max_time = 30.0
                    )
    x = copy(nlp.meta.x0)
    
    g(x) = grad(nlp, x)
    H(x) = Symmetric(hess(nlp, x), :L)
    t0 = time()
    Δt = 0.0
    
    k = 0
    gx = g(x)
    while !(norm(gx) < tol || k > kmax || Δt > max_time)
        d = -H(x) \ gx
        x = x + d
        gx = g(x)
        k = k + 1
        Δt = time() - t0 # Tempo decorrido
    end
    
    status = :unknown
    if norm(gx) < tol
        status = :first_order
    elseif k > kmax
        status = :max_iter
    elseif Δt > max_time
        status = :max_time
    end
    
    # Parte do SolverTools.jl
    return GenericExecutionStats(status, nlp, solution=x, objective=obj(nlp, x),
                                 dual_feas=norm(gx), iter=k, elapsed_time=Δt)
end

In [ ]:
output = newton_puro(nlp)
print(output)

## Coletâneas de problemas

Quando algoritmos eram criados há algumas décadas, era necessário criar também algumas funções para comparar esses algoritmos. Essas funções costumavam ser comparilhadas entre autores, para que todos pudessem fazer testes computacionais. Alguns artigos foram publicados descrevendo conjuntos de problemas que poderiam ser úteis em contextos específicos. Eis alguns:
- W. Hock, K. Schittkowski. **Test Examples for Nonlinear Programming Codes**, Springer, 1981
- Jorge J. Moré, Burton S. Garbow, and Kenneth E. Hillstrom. 1981. **Testing Unconstrained Optimization Software.** ACM Trans. Math. Softw. 7, 1 (1981), 17-41. DOI: https://doi.org/10.1145/355934.355936

Em 1995, com a publicação

- I. Bongartz, A. R. Conn, N. I. M. Gould, and Ph. L. Toint. CUTE: Constrained
and Unconstrained Testing Environment. ACM Transactions on Mathematical
Software, 21(1):123–160, 1995.

vários problemas foram colecionados num software que dava acesso à todas as qualidades de um problema de programação não-linear: função objetivo, restrições, gradientes, Hessianas, ponto inicial, etc. Essa biblioteca de testes, **CUTE** teve duas versões seguintes: **CUTEr** e **CUTEst**. Esta última, a mais atual, têm uma interface em Julia, que podemos acessar sem muita dificuldade.

## CUTEst.jl

Usar o `ADNLPModel` nos permite criar exemplos rápido, mas para testes computacionais de verdade, precisamos de uma biblioteca de testes. A mais usada está disponível no `CUTEst.jl`.

In [ ]:
using CUTEst

nlp = CUTEstModel("ROSENBR")
out = newton_puro(nlp)
finalize(nlp) # Necessário para fechar o problema
print(out)

O `CUTEst` tem todas as informações de problema armazenadas internamente, incluindo o ponto inicial, as funções e como calcular suas derivadas. Para acessá-las, é necessário saber o nome do problema. Como não sabemos isso, escolhemos problemas pelas suas características

In [ ]:
problemas = CUTEst.select(max_var=2, max_con=0, only_free_var=true)

In [ ]:
nlp = CUTEstModel("HILBERTA")
output = newton_puro(nlp)
println(output)
finalize(nlp)

In [ ]:
nlp = CUTEstModel("HIMMELBB")
local output
try
    output = newton_puro(nlp)
catch ex
    println("EXCECAO")
finally
    finalize(nlp)
end
println(output)

In [ ]:
gerador = (CUTEstModel(p) for p in problemas)
df = solve_problems(newton_puro, gerador)

In [ ]:
df

Agora vamos criar uma implementação do método do Gradiente com busca de Armijo

## Método do Gradiente com Busca Inexata

1. Dados $x_0$, $\varepsilon > 0$, $k = 0$, $k_\max \in \mathbb{N}$, $\alpha, \sigma \in (0,1)$.
2. Enquanto $\Vert\nabla f(x_k)\Vert > \varepsilon$
    1. $d_k = -\nabla f(x_k)$
    2. Defina $t_k$ como o primeiro valor da sequência
        $\{1,\sigma,\sigma^2,\sigma^3,\dots\}$ tal que
$$f(x_k + t_k d_k) < f(x_k) + \alpha t_k \nabla f(x_k)^Td_k. $$
    3. Calcule $x_{k+1} = x_k + t_k d_k$
    4. $k = k + 1$
    5. Teste outras condições de parada e vá à 4 se alguma for satisfeita
3. Fim do Enquanto
4. Saída: $x_k$, $f(x_k)$, $\Vert\nabla f(x_k)\Vert$, Tempo, Iterações, status

In [ ]:
function gradiente_armijo(nlp;
                          tol = 1e-6, kmax = 10000, max_time = 30.0,
                          α = 1e-4
                          )
    x = copy(nlp.meta.x0)
    
    f(x) = obj(nlp, x)
    g(x) = grad(nlp, x)
    
    t0 = time()
    Δt = 0.0
    
    status = :unknown
    k = 0
    fx = f(x)
    gx = g(x)
    while !(norm(gx) < tol || k > kmax || Δt > max_time)
        d = -gx
        t = 1.0
        xt = x + d
        ft = f(xt)
        while ft ≥ fx + α * t * dot(d, gx)
            t = 0.9t
            xt = x + t * d
            ft = f(xt)
            if t < 1e-16
                status = :stalled
                break
            end
        end
        if status == :stalled
            break
        end
        x = xt 
        fx = ft
        gx = g(x)
        k = k + 1
        Δt = time() - t0 # Tempo decorrido
    end
    
    if norm(gx) < tol
        status = :first_order
    elseif k > kmax
        status = :max_iter
    elseif Δt > max_time
        status = :max_time
    end
    
    # Parte do SolverTools.jl
    return GenericExecutionStats(status, nlp, solution=x, objective=obj(nlp, x),
                                 dual_feas=norm(gx), iter=k, elapsed_time=Δt)
end

In [ ]:
nlp = CUTEstModel("ROSENBR")
output = gradiente_armijo(nlp, α = 1e-2, kmax=100_000)
finalize(nlp)
println(output)

In [ ]:
nlp = CUTEstModel("HIMMELBB")
output = gradiente_armijo(nlp)
finalize(nlp)
println(output)

In [ ]:
nlp = CUTEstModel("HIMMELBB")
output = newton_puro(nlp)
finalize(nlp)
println(output)

# Comparações entre algoritmos de otimização

## Perfil de Desempenho

O perfil de desempenho é um gráfico de comparação de algoritmos útil quando existe uma troca de eficiência e robustez. Em geral ele não é muito útil para algoritmos que sempre convergem.

A ideia do perfil de desempenho é "normalizar" a comparação. Problemas menores tendem a ser resolvidos mais rápido, enquanto problemas maiores podem demorar vários minutos. Nessa situação, 1 minuto de diferença pode ser muito ou pouco.

In [ ]:
problemas = CUTEst.select(max_var=2, max_con=0, only_free_var=true)
gerador = (CUTEstModel(p) for p in problemas)
solvers = Dict(:newton_puro => newton_puro, :gradiente_armijo => gradiente_armijo)
stats = bmark_solvers(solvers, gerador)

In [ ]:
using SolverBenchmark

In [ ]:
function custo(df)
    falha = (df.status .!= :first_order)
    return falha * Inf + df.elapsed_time
end

In [ ]:
performance_profile(stats, custo)

In [ ]:
df_join = join(stats, [:status, :objective], invariant_cols=[:name])
print(df_join)

$$ f \le f_{\min} + 10^{-1}|f_{\min}| + 10^{-6} $$

In [ ]:
fmin = min.(stats[:newton_puro].objective, stats[:gradiente_armijo].objective)
function custo(df)
    falha = df.objective .≤ fmin + 1e-1 * abs.(fmin) .+ 1e-6
    return falha * Inf + df.elapsed_time
end

In [ ]:
performance_profile(stats, custo)

## Newton com Armijo e salvaguarda

- Gosto: $ \nabla^2 f(x^k) d = -\nabla f(x^k), $
- Preciso: $ \nabla^2 f(x^k)$ definida positiva
- Se não for, uso $d = -\nabla f(x^k)$.

In [ ]:
function newton_armijo(nlp;
                       tol = 1e-6, kmax = 10000, max_time = 30.0
                       )
    x = copy(nlp.meta.x0)
    
    f(x) = obj(nlp, x)
    g(x) = grad(nlp, x)
    H(x) = Symmetric(hess(nlp, x), :L)
    
    t0 = time()
    Δt = 0.0
    
    status = :unknown
    k = 0
    fx = f(x)
    gx = g(x)
    Hx = H(x)
    while !(norm(gx) < tol || k > kmax || Δt > max_time)
        d = -Hx \ gx
        if dot(d, gx) > 0
            d = -gx
        end
        t = 1.0
        xt = x + d
        ft = f(xt)
        while ft ≥ fx + 1e-4 * t * dot(d, gx)
            t = 0.9t
            xt = x + t * d
            ft = f(xt)
            if t < 1e-16
                status = :stalled
                break
            end
        end
        if status == :stalled
            break
        end
        x = xt 
        fx = ft
        gx = g(x)
        Hx = H(x)
        k = k + 1
        Δt = time() - t0 # Tempo decorrido
    end
    
    if norm(gx) < tol
        status = :first_order
    elseif k > kmax
        status = :max_iter
    elseif Δt > max_time
        status = :max_time
    end
    
    return GenericExecutionStats(status, nlp, solution=x, objective=obj(nlp, x),
                                 dual_feas=norm(gx), iter=k, elapsed_time=Δt)
end

In [ ]:
nlp = CUTEstModel("ROSENBR")
try
    output = newton_armijo(nlp)
    print(output)
catch ex
    println(ex)
finally
    finalize(nlp)
end

In [ ]:
solvers = Dict(:newton_puro => newton_puro,
               :newton_armijo => newton_armijo,
               :gradiente_armijo => gradiente_armijo)
stats = bmark_solvers(solvers, gerador)

In [ ]:
stats[:newton_armijo]

In [ ]:
performance_profile(stats, custo)

# Exemplo

In [ ]:
x = sort(rand(100))
y = [xi + randn() * 0.3 < 0.7 ? 0.0 : 1.0 for xi in x]
scatter(x, y, leg=false)

In [ ]:
σ(t) = 1 / (1 + exp(-t))
h(β, x) = σ(β[1] + β[2] * x)
scatter(x, y, leg=false)
plot!(x -> h([-5.0; 10.0], x), c=:red, lw=2)
plot!(x -> h([-3.0; 12.0], x), c=:green, lw=2)
plot!(x -> h([-3.0; 4.0], x), c=:yellow, lw=2)

$$ -y_i \ln h(\beta, x_i) - (1 - y_i) \ln \Big(1 - h(\beta, x_i)\Big) $$

In [ ]:
J(β) = sum(-y[i] * log(h(β, x[i])) - (1 - y[i]) * log(1 - h(β, x[i])) for i = 1:100)
nlp = ADNLPModel(J, ones(2))
output = newton_puro(nlp)
println(output)

In [ ]:
β = output.solution
scatter(x, y, leg=false)
plot!(x -> h(β, x), c=:red, lw=2)